## импорты

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from pymystem3 import Mystem
mystem_analyzer = Mystem(grammar_info=False, disambiguation=False)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
from collections import Counter

## импорт и обработка таблиц

In [ ]:
df = pd.read_excel("8.xlsx") # данные для обучения (уточнить адрес)
df2 = pd.read_excel("9.xlsx") # Данные для предсказания (уточнить адрес)

In [ ]:
# 1) при токенизации удаляются знаки препинания
# 2) Слова ставятся в начальную форму, записываются в строку через пробел
tokenizer = RegexpTokenizer(r'\w+')
df["текст"] = [tokenizer.tokenize(n) for n in df["текст"]]
df["текст"] = [' '.join(n) for n in df["текст"]]
df["текст"] = [mystem_analyzer.lemmatize(n) for n in df["текст"]]
# сохраняем копию списка лемматизированных слов
#vocab = df[["текст"]].copy() - из старого кода
df["текст"] = [''.join(n) for n in df["текст"]]

df2["текст"] = [tokenizer.tokenize(n) for n in df2["текст"]]
df2["текст"] = [' '.join(n) for n in df2["текст"]]
df2["текст"] = [mystem_analyzer.lemmatize(n) for n in df2["текст"]]
# сохраняем копию списка лемматизированных слов
#vocab2 = df2[["текст"]].copy() - из старого кода
df2["текст"] = [''.join(n) for n in df2["текст"]]

In [ ]:
X_pred_txt = df2["текст"]

## Визирование таблиц

In [ ]:
df.head(5)

In [ ]:
df2.head(5)

## Вычленение ключевых слов (выполнить 1 раз)

In [ ]:
words = []
for sentence in df["ключевые слова"].tolist():
    sentence = str(sentence).replace("\t", "") #удаляем лишнее из данных для обучения
    sentence = sentence.replace(",", ", ")
    sentence = sentence.replace("  ", " ")
    sentence = sentence.replace("  ", " ")
    sentence = sentence.split(", ") #разделяем набор ключевых слов
    for word in sentence: #добавляем в список уникальные значения ключевых слов
        words.append(word)

In [ ]:
c= Counter(words)

In [ ]:
#c.most_common()[:-400-1:-1]

In [ ]:
words = c.most_common(400)
words = [n[0] for n in words]
words[-1:-100:-1]

## функция предсказания для 1 ключевого слова

In [ ]:
def Klslovo(a):
    answerlist = []
    for n in df['ключевые слова'].tolist():
        x = a in str(n)
        answerlist.append(int(x))
    df["ответ"] = answerlist #создаём новую колонку в таблице: она содержит единицы в строках, где есть ключевое слово
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], \
                            df.iloc[:,-1].tolist(), \
                            test_size=0.1, \
                            random_state=42)
    X_train = X_train["текст"] # учимся на колонке с текстом
    X_test = X_test["текст"] # учимся на колонке с текстом
    pipeline = Pipeline([
        ('vect', CountVectorizer(ngram_range=(1, 2))),#уточнить, нужны ли н-граммы
        ('tfidf', TfidfTransformer()),
        ('clf', LogisticRegression(penalty="l2", class_weight="balanced", solver="liblinear")),
    ])
    pipeline.fit(X_train, y_train)
    prediction = pipeline.predict(X_test)
    print("\n" + "для слова " + a + " результаты прогноза:")
    precision = precision_score(y_true=y_test, y_pred=prediction, average="macro")
    print("precision = " + str(precision))
    recall = recall_score(y_true=y_test, y_pred=prediction, average="macro")
    print("recall = " + str(recall))
    accuracy = accuracy_score(y_true=y_test, y_pred=prediction)
    print("accuracy = " + str(accuracy))
    f1 = f1_score(y_true=y_test, y_pred=prediction, average="macro")
    print("f1 = " + str(f1))
    return a, pipeline.predict(X_pred_txt)

In [ ]:
#first_Word, first_Answer = Klslovo(words[0])
#for n in np.where(first_Answer == 1):
#    df2.loc[n, "ключевые слова"] = str(first_Word)

In [ ]:
#df2.head(20)

In [ ]:
resulti = []
for n in range(len(X_pred_txt)):
    resulti.append([])

## Функция записи ключевых слов в список

In [ ]:
def writedown(a, answers):
    for n in np.where(answers == 1)[0]:
        resulti[int(n)].append(a)
# остатки старого кода:
#        if type(df2.loc[n, "ключевые слова"]) is float == True:
#            df2.loc[n, "ключевые слова"] = str(a)
#            print("1")
#        elif type(df2.loc[n, "ключевые слова"]) is str == True:
#            df2.loc[n, "ключевые слова"] = df2.loc[n, "ключевые слова"] + ", " + a
#            print("2")

## Применение функций ко всем словам

In [ ]:
for n in words:
    results = Klslovo(n)
    writedown(results[0], results[1])

In [ ]:
resulti[:3]

In [ ]:
resulti2 = [", ".join(n) for n in resulti]
resulti2[:3]

## запись в таблицу

In [ ]:
df2["ключевые слова"] = pd.Series(resulti2)
df2.head(5)

## Сохранение таблицы

In [ ]:
df2.to_excel("13.xlsx")